In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [18]:
#смена ip
from random import randint
random = randint(120,140)
opts = Options()
opts.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random}.0.0.0 Safari/537.36")
ip = '133.18.234.13'
port = '80'
opts.add_argument(f"--proxy-server=http://{ip}:{port}")
driver = webdriver.Chrome(options=opts)
driver.get("https://2ip.ru/")

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()  # или Firefox, Edge и т.д.
driver.get("https://free-proxy-list.net/ru/")

table = driver.find_element(By.CSS_SELECTOR, value = '#list > div > div.table-responsive > div > table > tbody > tr:nth-child(1)').get_attribute('textContent')
rows = driver.find_elements(By.CSS_SELECTOR, "#list > div > div.table-responsive > div > table > tbody > tr")
table_data = []

for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    table_data.append([cell.text for cell in cells])

print(table_data)


[['222.252.194.29', '8080', 'VN', 'Vietnam', 'elite proxy', 'no', 'yes', '0 secs ago'], ['133.18.234.13', '80', 'JP', 'Japan', 'anonymous', '', 'no', '0 secs ago'], ['32.223.6.94', '80', 'US', 'United States', 'anonymous', 'no', 'no', '0 secs ago'], ['103.94.52.70', '3128', 'MM', 'Myanmar', 'elite proxy', 'no', 'no', '0 secs ago'], ['123.141.181.7', '5031', 'KR', 'South Korea', 'elite proxy', 'no', 'no', '0 secs ago'], ['50.122.86.118', '80', 'US', 'United States', 'anonymous', 'no', 'no', '0 secs ago'], ['200.174.198.158', '8888', 'BR', 'Brazil', 'anonymous', 'yes', 'yes', '0 secs ago'], ['4.156.78.45', '80', 'US', 'United States', 'anonymous', '', 'no', '0 secs ago'], ['185.36.145.215', '80', 'SE', 'Sweden', 'anonymous', 'no', 'no', '0 secs ago'], ['72.10.164.178', '2493', 'CA', 'Canada', 'elite proxy', '', 'no', '0 secs ago'], ['152.53.107.230', '80', 'NL', 'Netherlands', 'anonymous', 'no', 'no', '0 secs ago'], ['81.169.213.169', '8888', 'DE', 'Germany', 'anonymous', 'no', 'no', '0 

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


driver = webdriver.Chrome()
 
driver.get("https://free-proxy-list.net/ru/")
time.sleep(3)


headers = driver.find_elements(By.CSS_SELECTOR, "#list > div > div.table-responsive > div > table > thead > tr > th")
columns = [h.text.strip() for h in headers]

rows = driver.find_elements(By.CSS_SELECTOR, "#list > div > div.table-responsive > div > table > tbody > tr")

table_data = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    values = [cell.text.strip() for cell in cells]
    if values:
        table_data.append(values)

df = pd.DataFrame(table_data, columns=columns)

df_filtered = df[df["Anonymity"].str.lower() == "anonymous"]
df_final = df_filtered[["IP Address", "Port"]]

print(df_final)

df_final.to_csv("anonymous_proxies.csv", index=False, encoding="utf-8")

driver.quit()


AttributeError: 'NoneType' object has no attribute 'strip'

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException, TimeoutException, NoSuchElementException
import time
from random import randint


df_final = pd.read_csv("anonymous_proxies.csv")

working_proxies = []

for idx, row in df_final.iterrows():
    ip = row["IP Address"]
    port = row["Port"]

    try:
        random_version = randint(120, 140)
        opts = Options()
        opts.add_argument(
            f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            f"AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random_version}.0.0.0 Safari/537.36"
        )
        opts.add_argument(f"--proxy-server=http://{ip}:{port}")
        driver = webdriver.Chrome(options=opts)
        driver.set_page_load_timeout(10)

        driver.get("https://2ip.ru/")
        time.sleep(3)

        try:
            visible_ip = driver.find_element(By.CSS_SELECTOR, ".ip").text.strip()
        except NoSuchElementException:
            try:
                visible_ip = driver.find_element(By.CSS_SELECTOR, ".ip__value").text.strip()
            except:
                visible_ip = ""

        if visible_ip and visible_ip.startswith(ip.split(".")[0]):  

            print(f"OK {ip}:{port} — подтверждён ({visible_ip})")
            working_proxies.append((ip, port))
        else:
            print(f"FAIL {ip}:{port} — IP не совпадает (виден {visible_ip})")

    except (WebDriverException, TimeoutException) as e:
        print(f"FAIL {ip}:{port} — {e}")
    finally:
        try:
            driver.quit()
        except:
            pass

df_working = pd.DataFrame(working_proxies, columns=["IP Address", "Port"])
df_working.to_csv("working_proxies.csv", index=False, encoding="utf-8")

print("\n Рабочие прокси сохранены в working_proxies.csv")


FAIL 123.58.199.232:8168 — Message: unknown error: net::ERR_TUNNEL_CONNECTION_FAILED
  (Session info: chrome=140.0.7339.129)
Stacktrace:
	GetHandleVerifier [0x0x7ff7e49b1eb5+80197]
	GetHandleVerifier [0x0x7ff7e49b1f10+80288]
	(No symbol) [0x0x7ff7e47302fa]
	(No symbol) [0x0x7ff7e472d001]
	(No symbol) [0x0x7ff7e471da49]
	(No symbol) [0x0x7ff7e471f821]
	(No symbol) [0x0x7ff7e471dd66]
	(No symbol) [0x0x7ff7e471d7b7]
	(No symbol) [0x0x7ff7e471d47b]
	(No symbol) [0x0x7ff7e471afec]
	(No symbol) [0x0x7ff7e471b87c]
	(No symbol) [0x0x7ff7e47343ca]
	(No symbol) [0x0x7ff7e47d95de]
	(No symbol) [0x0x7ff7e47b037a]
	(No symbol) [0x0x7ff7e47d87fb]
	(No symbol) [0x0x7ff7e47b0153]
	(No symbol) [0x0x7ff7e4778b02]
	(No symbol) [0x0x7ff7e47798d3]
	GetHandleVerifier [0x0x7ff7e4c6e83d+2949837]
	GetHandleVerifier [0x0x7ff7e4c68c6a+2926330]
	GetHandleVerifier [0x0x7ff7e4c886c7+3055959]
	GetHandleVerifier [0x0x7ff7e49ccfee+191102]
	GetHandleVerifier [0x0x7ff7e49d50af+224063]
	GetHandleVerifier [0x0x7ff7e49baf6